In [1]:
import sympy as sp

In [2]:
import sympy as sp

# Define the dimensions
n = 2  # Example dimension for matrices

# Define real matrix symbols
U = sp.MatrixSymbol('U', n, n)
A = sp.MatrixSymbol('A', n, n)

# Convert MatrixSymbols to actual Matrices for operations
U = sp.Matrix(U, real=True)
A = sp.Matrix(A, real=True, symmetric=True)

# Perform matrix multiplication
UAU_T = U * A * U.T

# Extract diagonal elements
diag_UAU_T = sp.Matrix([UAU_T[i, i] for i in range(n)])

# Calculate Frobenius norm squared of the diagonal matrix
Err_U_A = diag_UAU_T.norm(2)**2

# Simplify the expression (optional)
Err_U_A_simplified = sp.simplify(Err_U_A)

# Compute the gradient of the error function with respect to each element of U
gradient_matrix = sp.Matrix(n, n, lambda i, j: Err_U_A.diff(U[i, j]))

print(gradient_matrix)

Matrix([[2*(((re(A[0, 0]*U[1, 0]) + re(A[1, 0]*U[1, 1]))*re(U[1, 0]) + (re(A[0, 1]*U[1, 0]) + re(A[1, 1]*U[1, 1]))*re(U[1, 1]) - (im(A[0, 0]*U[1, 0]) + im(A[1, 0]*U[1, 1]))*im(U[1, 0]) - (im(A[0, 1]*U[1, 0]) + im(A[1, 1]*U[1, 1]))*im(U[1, 1]))*((re(A[0, 0]*U[1, 0]) + re(A[1, 0]*U[1, 1]))*Derivative(re(U[1, 0]), U[0, 0]) + (re(A[0, 1]*U[1, 0]) + re(A[1, 1]*U[1, 1]))*Derivative(re(U[1, 1]), U[0, 0]) + (-im(A[0, 0]*U[1, 0]) - im(A[1, 0]*U[1, 1]))*Derivative(im(U[1, 0]), U[0, 0]) + (-im(A[0, 1]*U[1, 0]) - im(A[1, 1]*U[1, 1]))*Derivative(im(U[1, 1]), U[0, 0]) + (Derivative(re(A[0, 0]*U[1, 0]), U[0, 0]) + Derivative(re(A[1, 0]*U[1, 1]), U[0, 0]))*re(U[1, 0]) + (Derivative(re(A[0, 1]*U[1, 0]), U[0, 0]) + Derivative(re(A[1, 1]*U[1, 1]), U[0, 0]))*re(U[1, 1]) + (-Derivative(im(A[0, 0]*U[1, 0]), U[0, 0]) - Derivative(im(A[1, 0]*U[1, 1]), U[0, 0]))*im(U[1, 0]) + (-Derivative(im(A[0, 1]*U[1, 0]), U[0, 0]) - Derivative(im(A[1, 1]*U[1, 1]), U[0, 0]))*im(U[1, 1])) + ((re(A[0, 0]*U[1, 0]) + re(A[1, 0]

In [3]:
import numpy as np
n = 10
A = np.random.rand(n, n)
A = A @ A.T  # Make A symmetric positive definite

A_bk = A.copy()
print(A)
U = np.eye(n)

def ddiag(U, A):
    return np.linalg.norm(np.diag(U @ A @ U.T), ord=2)**2

print(ddiag(U, A))
print(A[0, 0]**2 + A[1, 1]**2 + A[2, 2]**2)

[[2.68570611 1.52013907 2.93022225 2.15497729 1.61277953 2.98289199
  2.46323561 2.02919724 2.27730742 1.88511059]
 [1.52013907 1.93761051 2.5816398  1.79635617 1.07822346 2.19405994
  1.75913631 1.22940814 2.05240709 1.84724538]
 [2.93022225 2.5816398  4.94408533 3.35443119 2.13195637 3.49040125
  3.7889721  2.7852824  3.48108624 3.01670073]
 [2.15497729 1.79635617 3.35443119 2.54139571 1.60310514 2.69239421
  2.53667253 1.92854455 2.09219126 1.81508761]
 [1.61277953 1.07822346 2.13195637 1.60310514 1.38449723 1.74737631
  1.34163659 1.09604698 1.52184428 1.40317709]
 [2.98289199 2.19405994 3.49040125 2.69239421 1.74737631 4.10176818
  3.19349438 2.48811673 2.78576751 2.52709147]
 [2.46323561 1.75913631 3.7889721  2.53667253 1.34163659 3.19349438
  4.01087666 3.03740275 3.05079653 2.53648626]
 [2.02919724 1.22940814 2.7852824  1.92854455 1.09604698 2.48811673
  3.03740275 2.55272449 2.3950626  1.89366306]
 [2.27730742 2.05240709 3.48108624 2.09219126 1.52184428 2.78576751
  3.05079653

In [4]:
from scipy.linalg import polar

def diag_norm(A):
    return np.linalg.norm(np.diag(A), ord=2)

def riemannian_diagonalization(A, U = None, h = 1e-2, iterations = 100, out = False):
    n = A.shape[1]
    if U == None:
        U = np.eye(n)

    A = A.copy()
    Utot = U.copy()

    d1 = diag_norm(A)
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        U = np.eye(n)
        for p in range(n):
            for q in range(n):
                U[p, q] -= h * A[p, p] * (A[p, q] + A[q, p])

        # projection to unitary space
        U, _ = polar(U, side = 'left')

        # A' = U^T A U
        A2 = U.T @ A @ U
        d2 = diag_norm(A2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-16:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    return np.diagonal(A), Utot

d, Utot = riemannian_diagonalization(A)

A2 = Utot.T @ A @ Utot
A2[np.abs(A2) < 1e-10] = 0

print(sorted(d))
A = A_bk.copy()
d2, U2 = np.linalg.eigh(A)
print(d2)
print(np.linalg.norm(d2 - sorted(d)))
print(Utot.T @ A @ Utot)


[np.float64(0.0015960360199368249), np.float64(0.023834448998615348), np.float64(0.1015485691505898), np.float64(0.17546396832392208), np.float64(0.38265480719656053), np.float64(0.7159280089070644), np.float64(1.1782772505717296), np.float64(1.7326450227651267), np.float64(1.8662531914840752), np.float64(24.7400843696787)]
[1.59603602e-03 2.38344490e-02 1.01548569e-01 1.75463968e-01
 3.82654807e-01 7.15928009e-01 1.17827725e+00 1.73264502e+00
 1.86625319e+00 2.47400844e+01]
6.091931331579728e-13
[[ 1.01548569e-01  4.25038161e-16 -9.72263589e-15  4.64779261e-16
   6.03265229e-16  1.60879870e-16 -9.40609120e-16 -4.59826858e-16
  -4.02315205e-16  7.84258720e-17]
 [ 8.78239505e-17  3.82654807e-01  9.79621083e-14  1.58729081e-15
  -1.45961298e-15 -3.77861047e-13 -1.81018805e-15  5.70498151e-15
   2.22184965e-15  6.77394409e-14]
 [-1.05074501e-14  9.82331457e-14  2.47400844e+01  5.84220156e-14
   6.56072779e-15  5.75939225e-14  5.00357337e-15  8.77156278e-14
   2.73722226e-14  4.69528105e-1

In [5]:
# Simultaneous diagonalization
from scipy.linalg import polar

def diag_norm_k(A):
    d = 0
    for i in range(A.shape[0]):
        d += np.linalg.norm(np.diag(A[i, :, :]), ord = 2)**2
    return d

A = np.random.rand(3, 3, 3)

def transform_matrices(A, U):
    A = A.copy()
    for k in range(A.shape[0]):
        A[k, :, :] = U.T @ A[k, :, :] @ U
    return A

def riemannian_joint_diagonalization(A, U = None, h = 1e-2, iterations = 100, out = False):
    if U == None:
        U = np.eye(A.shape[1])

    A = A.copy()
    Utot = U.copy()
    n = A.shape[1]

    d1 = diag_norm_k(A)
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        U = np.eye(n)
        for p in range(n):
            for q in range(n):
                for k in range(A.shape[0]):
                    U[p, q] -= h * A[k, p, p] * (A[k, p, q] + A[k, q, p])

        # projection to unitary space
        U, _ = polar(U, side = 'left')

        # A' = U^T A U
        A2 = transform_matrices(A, U)
        d2 = diag_norm_k(A2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-16:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    d = np.zeros([A.shape[1], A.shape[2]])
    for k in range(A.shape[0]):
        d[k, :] = np.diagonal(A[k, :, :])
    return d, Utot

d, Utot = riemannian_joint_diagonalization(A)
print(d)
A2 = transform_matrices(A, Utot)
print(A2)

[[ 1.39830547 -0.23178817 -0.25886501]
 [ 1.73100895  0.14827603  0.09227184]
 [ 1.78227417  0.66989061 -0.1764048 ]]
[[[ 1.39830547 -0.63843238  0.12861651]
  [ 0.55101254 -0.23178817  0.20355906]
  [ 0.02284089 -0.15729542 -0.25886501]]

 [[ 1.73100895  0.17411327  0.28572971]
  [-0.18249805  0.14827603  0.10471352]
  [-0.50082701  0.25805839  0.09227184]]

 [[ 1.78227417  0.07066892  0.11749516]
  [ 0.06936758  0.66989061 -0.06166549]
  [-0.06567458  0.03585114 -0.1764048 ]]]


In [6]:
# Weighted Simultaneous diagonalization
from scipy.linalg import polar
import numpy as np

def weighted_diagonal_norm(A, W):
    d = 0
    for k in range(A.shape[0]):
        for i in range(A.shape[1]):
            d += A[k, i, i]**2 / W[i, i]
    return d

def transform_matrices_w(A, W, U):
    A = A.copy()
    for k in range(A.shape[0]):
        A[k, :, :] = U.T @ A[k, :, :] @ U
    W = U.T @ W @ U
    return A, W

def weighted_diagonals(A, W):
    d = np.zeros([A.shape[1], A.shape[2]])
    for k in range(A.shape[0]):
        d[k, :] = np.diagonal(A[k, :, :]) / np.diagonal(W)
    return d

def weighted_symmetric(A, W):
    for k in range(A.shape[0]):
        A[k, :, :] = 0.5 * (A[k, :, :] @ W + W @ A[k, :, :])
    return A

def riemannian_joint_diagonalization(A, W, U = None, h = 1e-3, iterations = 1000, out = False):
    if U == None:
        U = np.eye(A.shape[1])

    A = A.copy()
    Utot = U.copy()
    n = A.shape[1]

    d1 = weighted_diagonal_norm(A, W)
    d0 = d1
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        G = np.zeros([n, n])
        G2 = np.zeros([n, n])
        for p in range(n):
            for q in range(n):
                for k in range(A.shape[0]):
                    G[p, q] += 2 * A[k, q, q] / W[q, q] * A[k, q, p] - A[k, q, q]**2 / W[q, q]**2 * W[q, p]
#                    G[p, q] += A[k, q, q] / W[q, q] * 2 * A[k, q, p] #- A[k, q, q]**2 / W[q, q]**2 * W[q, p] # like equations but seemingly problematic due to diagonal elements q, q
                    #G[p, q] += A[k, p, p] / W[p, p] * 2 * A[k, p, q] - A[k, p, p]**2 / W[p, p]**2 * W[p, q] # corrected version based on gut-feeling. multiplying second part with <1. seems to make it work
#                    G[p, q] += A[k, p, p] / W[p, p] * (2 * A[k, p, q] - 0.95 * A[k, p, p] * W[p, q] / W[p, p])
#                    G2[p, q] += A[k, p, p] * (A[k, p, q] + A[k, q, p])
                    G2[p, q] += A[k, q, q] * (A[k, q, p] + A[k, p, q])

#        if it % 1 == 0:
#            print(it, 'G-G2 = ', np.diag(G-G2)/np.diag(G))
#            print(it, 'G = ', G)
#            print(it, 'G2 = ', G2, '\n')
        U = np.eye(n)
        U += h * G

        # projection to unitary space
        U, _ = polar(U)
#        print('U=\n',U)
        U2 = np.eye(n) + h * G2
        U2, _ = polar(U2)
        U = U2
#        print('U2=\n',U2)

        # A' = U^T A U
        A2, W2 = transform_matrices_w(A, W, U)
        d2 = weighted_diagonal_norm(A2, W2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            W = W2.copy()
            if h < 1e1:
                h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-15:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    d = weighted_diagonals(A, W)
    print('improvement: ', d1 / d0, '; d0 = ', d0, '; d1 = ', d1)
    print('convergence information: h = ', h, ', it = ', it)
    return d, Utot

def weighted_diag(A, W, U = None, h = 1e-1, micro_iterations = 100, macro_iterations = 3, out = False):
    Utot = np.eye(A.shape[1])
    for it in range(macro_iterations):
        d, U = riemannian_joint_diagonalization(A, W, h = 1e-2, iterations = micro_iterations, out = False)
        Utot = Utot @ U
        A, W = transform_matrices_w(A, W, U)
    return d, Utot
    

#def riemannian_joint_diagonalization(A, W, U = None, h = 1e-1, iterations = 1000, out = False):

def initial_A(n):
    A = np.random.rand(n, n, n)
    for k in range(n):
        A[k, :, :] = A[k, :, :] @ A[k, :, :].T
    A[1, :, :] = A[0, :, :] @ A[0, :, :]
    A[2, :, :] = A[0, :, :] @ A[1, :, :]
    for k in range(n):
        A[k, :, :] /= np.trace(A[k, :, :])
    return A

def initial_W(n):
    W = np.random.rand(n, n)
    W = W @ W.T
    for i in range(n):
        W[i, i] *= 5
    W /= np.trace(W)
    return W

def report(A, W, pre = None):
    print(pre + 'A =\n', A)
    print(pre + 'W =\n', W, np.trace(W))
    d = weighted_diagonals(A, W)
    print(pre + 'd =\n', d)

#W = initial_W(3)
W = np.eye(3)
A = initial_A(3)
A = weighted_symmetric(A, W)

report(A, W, pre = 'start ')

#d, Utot = riemannian_joint_diagonalization(A, W, out = False)
d, Utot = weighted_diag(A, W, macro_iterations = 5, micro_iterations=200)

A2, W2 = transform_matrices_w(A, W, Utot)

report(A2, W2, pre = 'end ')

start A =
 [[[0.17363727 0.25427995 0.24732417]
  [0.25427995 0.41709071 0.39887902]
  [0.24732417 0.39887902 0.40927202]]

 [[0.16567222 0.26433077 0.26085942]
  [0.26433077 0.42244818 0.41690469]
  [0.26085942 0.41690469 0.4118796 ]]

 [[0.16553194 0.26447158 0.2611143 ]
  [0.26447158 0.42255876 0.41719645]
  [0.2611143  0.41719645 0.4119093 ]]]
start W =
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 3.0
start d =
 [[0.17363727 0.41709071 0.40927202]
 [0.16567222 0.42244818 0.4118796 ]
 [0.16553194 0.42255876 0.4119093 ]]
improvement:  2.618865549519899 ; d0 =  1.1227987022494017 ; d1 =  2.9404588403666088
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.0000000405281033 ; d0 =  2.940458840366682 ; d1 =  2.940458959537902
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.000000048402274 ; d0 =  2.9404589595379473 ; d1 =  2.9404591018628476
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.0000000576280246 ; d0 